In [9]:
import pandas as pd
import numpy as np

import time
import pickle

In [10]:
import black
import jupyter_black

jupyter_black.load(line_length=79)

In [11]:
# FIX VSCODE PATH PROBLEM -- DELETE AFTER
import sys

# add path
sys.path.append(
    "/gpfs01/berens/user/rgonzalesmarquez/phd/chatgpt-excess-words/scripts/"
)
print(sys.path)

['/gpfs01/berens/user/rgonzalesmarquez', '/.pyenv/versions/miniconda3-latest/lib/python311.zip', '/.pyenv/versions/miniconda3-latest/lib/python3.11', '/.pyenv/versions/miniconda3-latest/lib/python3.11/lib-dynload', '', '/gpfs01/berens/user/rgonzalesmarquez/.local/lib/python3.11/site-packages', '/gpfs01/berens/user/rgonzalesmarquez/phd/text-embeddings', '/gpfs01/berens/user/rgonzalesmarquez/phd/pubmed-landscape', '/.pyenv/versions/miniconda3-latest/lib/python3.11/site-packages', '/gpfs01/berens/user/rgonzalesmarquez/phd/chatgpt-excess-words/scripts/', '/gpfs01/berens/user/rgonzalesmarquez/phd/chatgpt-excess-words/scripts/']


In [63]:
%load_ext autoreload
%autoreload 2

from daily_updates_functions import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


We extract from PubMed's metadata `.xml` files the following:
- PubMed ID
- title
- abstract
- language
- journal title
- ISSN
- publication date
- (first and last) author first names
- (first and last) author Affiliations


# Parse data

## 2024 daily updates

In [16]:
%%time

path = "/gpfs01/berens/data/data/pubmed/2024_daily_updates/"

files_2024_df_daily_updates_v1 = import_all_files(path, order_files=True)


pubmed24n1220.xml
pubmed24n1221.xml
pubmed24n1222.xml
pubmed24n1223.xml
pubmed24n1224.xml
pubmed24n1225.xml
pubmed24n1226.xml
pubmed24n1227.xml
pubmed24n1228.xml
pubmed24n1229.xml
pubmed24n1230.xml
pubmed24n1231.xml
pubmed24n1232.xml
pubmed24n1233.xml
pubmed24n1234.xml
pubmed24n1235.xml
pubmed24n1236.xml
pubmed24n1237.xml
pubmed24n1238.xml
pubmed24n1239.xml
pubmed24n1240.xml
pubmed24n1241.xml
pubmed24n1242.xml
pubmed24n1243.xml
pubmed24n1244.xml
pubmed24n1245.xml
pubmed24n1246.xml
pubmed24n1247.xml
pubmed24n1248.xml
pubmed24n1249.xml
pubmed24n1250.xml
pubmed24n1251.xml
pubmed24n1252.xml
pubmed24n1253.xml
pubmed24n1254.xml
pubmed24n1255.xml
pubmed24n1256.xml
pubmed24n1257.xml
pubmed24n1258.xml
pubmed24n1259.xml
pubmed24n1260.xml
pubmed24n1261.xml
pubmed24n1262.xml
pubmed24n1263.xml
pubmed24n1264.xml
pubmed24n1265.xml
pubmed24n1266.xml
pubmed24n1267.xml
pubmed24n1268.xml
pubmed24n1269.xml
pubmed24n1270.xml
pubmed24n1271.xml
pubmed24n1272.xml
pubmed24n1273.xml
pubmed24n1274.xml
pubmed24n1

In [17]:
print("There are {} papers".format(files_2024_df_daily_updates_v1.shape[0]))

There are 2925353 papers


In [18]:
files_2024_df_daily_updates_v1.head()

,PMID,Title,AbstractText,Language,Journal,Date,NameFirstAuthor,NameLastAuthor,ISSN,AffiliationFirstAuthor,AffiliationLastAuthor,filename
0,1025,Isolation and purification of cytokinin bindin...,,eng,Biochemical and biophysical research communica...,1975 Nov 17,T,K,0006-291X,,,pubmed24n1220.xml
1,1250,Purification and properties of isoenzymes of c...,Two isoenzymes of an NADP+ -dependent cinnamyl...,eng,European journal of biochemistry,1975 Nov 01,D,H,0014-2956,,,pubmed24n1220.xml
2,433,"Uptake, translocation, and metabolism of tirpa...",,eng,Journal of agricultural and food chemistry,1975 Nov-Dec,J E,R I,0021-8561,,,pubmed24n1220.xml
3,1314,The soybean answer to food costs.,,eng,Dimensions in health service,1976 Jan,T C,T C,0317-7645,,,pubmed24n1220.xml
4,3,Metal substitutions incarbonic anhydrase: a ha...,,eng,Biochemical and biophysical research communica...,1975 Oct 27,R J,R G,0006-291X,,,pubmed24n1220.xml


# Filter

Filter out:
- non-English papers
- papers with empty abstracts
- papers with abstracts shorter than 250 or longer than 4000 symbols
- papers with unfinished abstracts

## Empty abstracts and non-english papers

In [21]:
%%time

print(
    "Before, there were {} papers".format(
        files_2024_df_daily_updates_v1.shape[0]
    )
)

# Eliminate empty abstracts
clean_2024_df_daily_updates_v1 = files_2024_df_daily_updates_v1[
    files_2024_df_daily_updates_v1.AbstractText != ""
]

print(
    "After eliminating empty abstracts, there are {} papers".format(
        clean_2024_df_daily_updates_v1.shape[0]
    )
)

# Eliminate non-english papers
clean_2024_df_daily_updates_v1 = clean_2024_df_daily_updates_v1[
    clean_2024_df_daily_updates_v1.Language == "eng"
]

# print size
print(
    "After first cleaning, there are {} papers".format(
        clean_2024_df_daily_updates_v1.shape[0]
    )
)

Before, there were 2925353 papers
After eliminating empty abstracts, there are 2621886 papers
After first cleaning, there are 2591305 papers
CPU times: user 3.9 s, sys: 288 ms, total: 4.19 s
Wall time: 4.14 s


## Threshold and cut off

### Cut off = 4000

In [22]:
print(
    "Before cut off, there are {} papers".format(
        clean_2024_df_daily_updates_v1.shape[0]
    )
)
abstracts = clean_2024_df_daily_updates_v1["AbstractText"].tolist()
len_strings = map(len, abstracts)
len_abstracts = np.fromiter(len_strings, dtype=np.int64, count=len(abstracts))


cut_off = 4000
clean_2024_df_daily_updates_v1 = clean_2024_df_daily_updates_v1[
    len_abstracts < cut_off
]
print(
    "After cut off, there are {} papers".format(
        clean_2024_df_daily_updates_v1.shape[0]
    )
)

Before cut off, there are 2591305 papers
After cut off, there are 2586634 papers


### Threshold = 250

In [23]:
threshold = 250
len_short_abstracts = len_abstracts[len_abstracts < cut_off]
clean_2024_df_daily_updates_v1 = clean_2024_df_daily_updates_v1[
    len_short_abstracts > threshold
]
print(
    "After threshold, there are {} papers".format(
        clean_2024_df_daily_updates_v1.shape[0]
    )
)

After threshold, there are 2567206 papers


## Remove the truncated sentence from abstracts


In [24]:
abstracts = clean_2024_df_daily_updates_v1["AbstractText"]
abstracts_list = clean_2024_df_daily_updates_v1["AbstractText"].tolist()

In [25]:
%%time
# BUENO
clean_2024_df_daily_updates_v1.AbstractText = list(
    map(
        lambda x, y: x[: y - 1] if y != -1 else x,
        clean_2024_df_daily_updates_v1.AbstractText,
        clean_2024_df_daily_updates_v1.AbstractText.str.find(
            "ABSTRACT TRUNCATED AT"
        ),
    )
)

CPU times: user 3.88 s, sys: 130 ms, total: 4.01 s
Wall time: 4.01 s


## Remove unfinished abstracts

In [26]:
abstracts = clean_2024_df_daily_updates_v1["AbstractText"]
abstracts_list = clean_2024_df_daily_updates_v1["AbstractText"].tolist()
print(len(abstracts_list))

2567206


In [27]:
%%time
end_abstracts = [x[-2:] for x in abstracts_list]

CPU times: user 556 ms, sys: 9.86 ms, total: 565 ms
Wall time: 564 ms


In [28]:
%%time
point_index = np.array([x.find(".") for x in end_abstracts])
question_index = np.array([x.find("?") for x in end_abstracts])
exclamation_index = np.array([x.find("!") for x in end_abstracts])

CPU times: user 1.25 s, sys: 2.99 ms, total: 1.26 s
Wall time: 1.25 s


In [29]:
%%time

print(
    "Before cleaning, there are {} papers".format(
        clean_2024_df_daily_updates_v1.shape[0]
    )
)

# Eliminate unfinished abstracts
clean_2024_df_daily_updates_v1 = clean_2024_df_daily_updates_v1[
    (point_index != -1) | (question_index != -1) | (exclamation_index != -1)
]

# print size
print(
    "After cleaning, there are {} papers".format(
        clean_2024_df_daily_updates_v1.shape[0]
    )
)

Before cleaning, there are 2567206 papers
After cleaning, there are 2566412 papers
CPU times: user 980 ms, sys: 52.1 ms, total: 1.03 s
Wall time: 1.03 s


# Generate labels

In [33]:
label_color_legend = {
    "cancer": "black",
    "neuroscience": "#aeaa00",
    "cardiology": "#1CE6FF",
    "ecology": "#FF34FF",
    "bioinformatics": "#FF4A46",
    "chemistry": "#008941",
    "surgery": "#006FA6",
    "environment": "#0089A3",
    "material": "#0000A6",
    "microbiology": "#B79762",
    "pediatric": "#004D43",
    "immunology": "#8FB0FF",
    "psychology": "#5A0007",
    "psychiatry": "#BA0900",
    "genetics": "#1B4400",
    "nutrition": "#4FC601",
    "veterinary": "#3B5DFF",
    "engineering": "#00C2A0",
    "education": "#549E79",
    "physics": "#BC23FF",
    "optics": "#C895C5",
    "nursing": "#FF2F80",
    "neurology": "#009271",
    "radiology": "#00FECF",
    "ophthalmology": "#A4E804",
    "gynecology": "#FFB500",
    "rehabilitation": "#6B002C",
    "pathology": "#FF9408",
    "anesthesiology": "#CC0744",
    "dermatology": "#D790FF",
    "pharmacology": "#5B4534",
    "physiology": "#E83000",
    "virology": "#6F0062",
    "biochemistry": "#b65141",
    "computation": "#C20078",
    "infectious": "#7A4900",
    "healthcare": "#FF90C9",
    "ethics": "#6508ba",
    "dentistry": "#8e4d8a",
}

In [34]:
%%time
labels_2024_du, _ = improved_coloring(
    clean_2024_df_daily_updates_v1.Journal, label_color_legend
)

CPU times: user 2min 39s, sys: 2 s, total: 2min 41s
Wall time: 2min 41s


# Extract first author's affiliation country

In [35]:
all_countries = [
    "Afghanistan",
    "Albania",
    "Algeria",
    "Andorra",
    "Angola",
    "Antigua and Barbuda",
    "Argentina",
    "Armenia",
    "Australia",
    "Austria",
    "Azerbaijan",
    "Bahamas",
    "Bahrain",
    "Bangladesh",
    "Barbados",
    "Belarus",
    "Belgium",
    "Belize",
    "Benin",
    "Bhutan",
    "Bolivia",
    "Bosnia and Herzegovina",
    "Botswana",
    "Brazil",
    "Brunei",
    "Bulgaria",
    "Burkina Faso",
    "Burundi",
    "Cabo Verde",
    "Cambodia",
    "Cameroon",
    "Canada",
    "Central African Republic",
    "Chad",
    "Chile",
    "China",
    "Colombia",
    "Comoros",
    "Democratic Republic of the Congo",
    "Republic of the Congo",
    "Costa Rica",
    "Côte d’Ivoire",
    "Croatia",
    "Cuba",
    "Cyprus",
    "Czech Republic",
    "Denmark",
    "Djibouti",
    "Dominica",
    "Dominican Republic",
    "East Timor",
    "Ecuador",
    "Egypt",
    "El Salvador",
    "Equatorial Guinea",
    "Eritrea",
    "Estonia",
    "Eswatini",
    "Ethiopia",
    "Fiji",
    "Finland",
    "France",
    "Gabon",
    "Gambia",
    "Georgia",
    "Germany",
    "Ghana",
    "Greece",
    "Grenada",
    "Guatemala",
    "Guinea",
    "Guinea-Bissau",
    "Guyana",
    "Haiti",
    "Honduras",
    "Hungary",
    "Iceland",
    "India",
    "Indonesia",
    "Iran",
    "Iraq",
    "Ireland",
    "Israel",
    "Italy",
    "Jamaica",
    "Japan",
    "Jordan",
    "Kazakhstan",
    "Kenya",
    "Kiribati",
    "North Korea",
    "South Korea",
    "Kosovo",
    "Kuwait",
    "Kyrgyzstan",
    "Laos",
    "Latvia",
    "Lebanon",
    "Nicaragua",
    "Niger",
    "Nigeria",
    "North Macedonia",
    "Norway",
    "Oman",
    "Pakistan",
    "Palau",
    "Panama",
    "Papua New Guinea",
    "Paraguay",
    "Peru",
    "Philippines",
    "Poland",
    "Portugal",
    "Qatar",
    "Romania",
    "Russia",
    "Rwanda",
    "Saint Kitts and Nevis",
    "Saint Lucia",
    "Saint Vincent and the Grenadines",
    "Samoa",
    "San Marino",
    "Sao Tome and Principe",
    "Saudi Arabia",
    "Senegal",
    "Serbia",
    "Seychelles",
    "Sierra Leone",
    "Singapore",
    "Slovakia",
    "Slovenia",
    "Solomon Islands",
    "Somalia",
    "South Africa",
    "Spain",
    "Sri Lanka",
    "Sudan",
    "South Sudan",
    "Suriname",
    "Sweden",
    "Switzerland",
    "Syria",
    "Taiwan",
    "Tajikistan",
    "Tanzania",
    "Thailand",
    "Togo",
    "Tonga",
    "Trinidad and Tobago",
    "Tunisia",
    "Turkey",
    "Turkmenistan",
    "Tuvalu",
    "Uganda",
    "Ukraine",
    "United Arab Emirates",
    "United Kingdom",
    "United States",
    "Uruguay",
    "Uzbekistan",
    "Vanuatu",
    "Vatican City",
    "Venezuela",
    "Vietnam",
    "Yemen",
    "Zambia",
    "Zimbabwe",
]

In [36]:
dict_countries = dict(zip(all_countries, np.arange(1, len(all_countries) + 1)))

## Country mapping

In [37]:
%%time
countries_first_author_2024_du, _ = mapping_countries(
    clean_2024_df_daily_updates_v1.AffiliationFirstAuthor, dict_countries
)

CPU times: user 12min 20s, sys: 6.32 s, total: 12min 27s
Wall time: 12min 27s


## State mapping

In [38]:
all_states = [
    "Alabama",
    "Alaska",
    "Arizona",
    "Arkansas",
    "California",
    "Colorado",
    "Connecticut",
    "Delaware",
    "Florida",
    "Georgia",
    "Hawaii",
    "Idaho",
    "Illinois",
    "Indiana",
    "Iowa",
    "Kansas",
    "Kentucky",
    "Louisiana",
    "Maine",
    "Maryland",
    "Massachusetts",
    "Michigan",
    "Minnesota",
    "Mississippi",
    "Missouri",
    "Montana",
    "Nebraska",
    "Nevada",
    "New Hampshire",
    "New Jersey",
    "New Mexico",
    "New York",
    "North Carolina",
    "North Dakota",
    "Ohio",
    "Oklahoma",
    "Oregon",
    "Pennsylvania",
    "Rhode Island",
    "South Carolina",
    "South Dakota",
    "Tennessee",
    "Texas",
    "Utah",
    "Vermont",
    "Virginia",
    "Washington",
    "West Virginia",
    "Wisconsin",
    "Wyoming",
]

In [39]:
len(all_states)

50

In [43]:
dict_all_states = dict(zip(all_states, np.arange(1, len(all_states) + 1)))

In [41]:
%%time
labels_usa_states_first_author_2024_du, _ = mapping_states(
    clean_2024_df_daily_updates_v1.AffiliationFirstAuthor, dict_all_states
)

CPU times: user 3min 37s, sys: 1.87 s, total: 3min 39s
Wall time: 3min 39s


### Correct USA country with states

In [46]:
countries_first_author_2024_usa_corrected = np.where(
    labels_usa_states_first_author_2024_du != "unknown",
    "United States",
    countries_first_author_2024_du,
)

In [48]:
print(
    "Percentage of US papers before correcting",
    np.sum(countries_first_author_2024_du == "United States")
    / countries_first_author_2024_du.shape[0]
    * 100,
)
print(
    "Percentage of US papers after correcting",
    np.sum(countries_first_author_2024_usa_corrected == "United States")
    / countries_first_author_2024_du.shape[0]
    * 100,
)

Percentage of US papers before correcting 21.641887584690224
Percentage of US papers after correcting 26.0119575500738


# Save df


- PMID
- abstract
- journal
- publication year (update also preprocess-and-count bc the year is being extracted there)
- label
- affiliation country (from the first affiliation of the first author).

Also eliminate duplicates PMIDs `df = df.groupby(['PMID']).last()`

In [49]:
df = clean_2024_df_daily_updates_v1[
    ["PMID", "AbstractText", "Journal", "Date"]
].copy(deep=True)

In [50]:
df["Year"] = df.Date.str.extract("([12]\d\d\d)").values.astype(int)


In [53]:
df.pop("Date")

1           1975 Nov 01
6              1975 Dec
9              1975 Dec
11          1977 May 11
12             1977 Feb
               ...     
2925347     2024 Apr 18
2925348    2024 Jan-Dec
2925350     2024 Apr 18
2925351     2024 Apr 18
2925352     2024 Apr 18
Name: Date, Length: 2566412, dtype: object

In [55]:
df["Labels"] = labels_2024_du

In [56]:
df["Countries"] = countries_first_author_2024_usa_corrected

In [57]:
df.head()

,PMID,AbstractText,Journal,Year,Labels,Countries
1,1250,Two isoenzymes of an NADP+ -dependent cinnamyl...,European journal of biochemistry,1975,biochemistry,unknown
6,1467,Aphids transmitted poly-L-ornithine (PLO)-trea...,The Journal of general virology,1975,virology,unknown
9,1547,The weakly acidic fraction (WAF) of cigarette ...,Journal of the National Cancer Institute,1975,cancer,unknown
11,15593,Three sites of silicomolybdate reduction in th...,Biochimica et biophysica acta,1977,unlabeled,unknown
12,14954,1. A stable ATPase [EC 3.6.1.3] complex (TF0-F...,Journal of biochemistry,1977,biochemistry,unknown


In [58]:
len(df)

2566412

In [59]:
df = df.groupby(["PMID"]).last()

In [60]:
len(df)

1524848

In [62]:
%%time
compression_opts = dict(
    method="zip", archive_name="pubmed_daily_updates_2024_v1.csv"
)

df.to_csv(
    "/gpfs01/berens/user/rgonzalesmarquez/phd/chatgpt-excess-words/data/pubmed_daily_updates_2024_v1.zip",
    index=False,
    compression=compression_opts,
)

CPU times: user 2min 52s, sys: 3.18 s, total: 2min 55s
Wall time: 2min 55s


From 2024 zenodo version, 
drop: "x", "y", "Colors", "Retractions", "Country colors"
add (to 2024 baseline and daily updates): genders first and last authors, have to be imported as separate arrays () 